In [ ]:
class DAE(nn.Module):
    def __init__(self):
        super(DAE, self).__init__()
        vgg_encoder = vgg16(pretrained=True)  #I use VGG16 as an encoder to learn important features of images
        self.encoder = nn.Sequential(*list(vgg_encoder.features.children())[:-1])
        
        self.decoder = nn.ModuleList([
            nn.ConvTranspose2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1, output_padding=1),
            nn.Sigmoid(),  # Use sigmoid activation to ensure values are in the range [0, 1]
        ])
​
        
    def forward(self, x):
        x = self.encoder(x)
        for layer in self.decoder:
            x = layer(x)
        x = torch.nn.functional.interpolate(x, size=(150, 150), mode='bilinear', align_corners=False)
​
        return x


model = DAE().to(device)
summary(model)


#Loss function and optimizer

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=3e-4, weight_decay=1e-5)

